In [1]:
import os
import io
from copy import deepcopy
from pathlib import Path
from PIL import Image as PILImage
import cv2
import numpy as np

from IPython.display import display, Audio
from ipywidgets import widgets, interact

from utils.photo_utils import exif_rotate, get_photo, get_centermost_detections
from cloud_ocr.google_api import save_cuts_for_detection, detect_text, draw_ocr
import cloud_ocr.ocr_utils as ocr_fcns

home_dir = os.getcwd()
print("Welcome!")

Welcome!


In [2]:
# TODO: make it simple, this is workaround
os.chdir(Path(home_dir, "yolov5"))
from model import Model
os.chdir(home_dir)

In [3]:
# pricetag model
yolo_pricetag_weight_path = str(Path("yolov5", "weights", "pricetag_small.pt"))
yolo_pricetag_model = Model(yolo_pricetag_weight_path, device="cpu")  # ~ 0.2s on CPU 

# product name and price model
yolo_name_price_weight_path = str(Path("yolov5", "weights", "kaufland_name_price_small_320.pt"))
yolo_name_price_model = Model(yolo_name_price_weight_path, device="cpu")  # ~ 0.2s on CPU 
yolo_name_price_model.img_size = 320

print("Models loaded")

Models loaded


In [4]:
nums = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]
pers = [".", ",", ":"]

In [11]:
def callback(value):
    if btn_upload.data:

        # =============================================================================
        # INITIALIZE IMAGE
        # =============================================================================

        print("You upload a file:")
        im_file = get_photo(btn_upload, False)
        
        # =============================================================================
        # DETECT PRICES
        # =============================================================================
        try:
            det_pricetags = yolo_pricetag_model.detect([np.array(im_file)],
                                                       min_confidence=0.75,
                                                       mode="return")
            
            # =============================================================================
            # CUT PRICES
            # =============================================================================        

            # returns list of n centermost pricetags. To return only coordinates, use mode="return"
            print(f" Nalezeno {len(det_pricetags['cls'])} cenovek, čtu cenovku nejblíže středu snímku.")

            # returns list of n centermost pricetags. To return only coordinates, use mode="return" 
            cuts = get_centermost_detections(im_file,
                                             det_pricetags,
                                             n=1,
                                             mode="cut",
                                             bigger_box_by_pct=0.01
                                             )

            display(PILImage.fromarray(cuts[0]))
            
            # TODO: filter pricetag and read it
            
            # =============================================================================
            # READ PRICETAGS
            # =============================================================================   
            
            # saving cuts to be able to run Google API detection (custom load)
            cuts_path = save_cuts_for_detection(cuts)

            # run Google API text detection and OCR
            cuts_ocr = [detect_text(path) for path in cuts_path]

            # =============================================================================
            # CLASSIFY DETECTIONS
            # =============================================================================   
            
            # detect price and product name with yolo model, only the first pricetag
            det_name_price = yolo_name_price_model.detect([cuts[0]],
                                                          min_confidence=0.1,
                                                          mode="return")

            # find price by intersectiong google results with yolo detections
            price = ocr_fcns.get_overlaps_yolo_google(
                google_detections=cuts_ocr[0],
                yolo_detections=det_name_price,
                yolo_classes=yolo_name_price_model.names,
                what_to_find="price",
                what_to_ignore="product_name",
                n_to_return=2
            )
            
            price = " ". join(price[0])

            # find product_name similarly
            product_name = ocr_fcns.get_overlaps_yolo_google(
                google_detections=cuts_ocr[0],
                yolo_detections=det_name_price,
                yolo_classes=yolo_name_price_model.names,
                what_to_find="product_name",
                what_to_ignore="price",
                n_to_return=2
            )

            # filter price value
            cena = ""
            # starting to add nums and pers until there is another letter -> break the for cycle
            for val in price:
                # we want to start with a num
                if val in nums:
                    cena += val
                # if there is a "." for example add there are already some num added, add it as well
                elif val in pers and len(cena) > 0:
                    cena += val
                # if nothing was added yet, continue
                elif not cena:
                    continue
                # else break the cycle
                else:
                    break

            # print results 
            print(" ".join(product_name[0]))      
            print(cena)
        except Exception as e:
            print(e)
            print("Nedokážu přečíst obrázek, zkuste to prosím znovu.")
        
    return

In [9]:
btn_upload = widgets.FileUpload()

In [10]:
_ = interact(callback, value=btn_upload)

interactive(children=(FileUpload(value={}, description='Upload'), Output()), _dom_classes=('widget-interact',)…